In [1]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup

In [2]:
url='https://turbo.az/'

In [3]:
page=requests.get(url)

In [4]:
html=BeautifulSoup(page.content, 'html.parser')

In [5]:
cars_info=html.find_all('div', class_='products-i__bottom')

In [6]:
information=re.sub('<[^>]+>', ' ', str(cars_info))

In [7]:
information

'[   42 540  AZN    Kia Sorento  2015, 2.0 L, 151 333 km  Bakı, bugün 03:51  ,    64 400  AZN    Toyota RAV 4  2022, 2.0 L, 0 km  Bakı, bugün 10:35  ,    56 400  €    Alfa Romeo Stelvio  2022, 2.0 L, 0 km  Bakı, 14.03.2023 10:21  ,    46 900  $    Nissan Xterra  2022, 2.5 L, 0 km  Bakı, dünən 14:07  ,    38 500  $    BMW M5  2012, 4.4 L, 119 500 km  Bakı, 23.03.2023 09:55  ,    28 900  AZN    LADA (VAZ) Niva Bronto  2023, 1.7 L, 0 km  Bakı, 30.03.2023 18:08  ,    50 300  $    Toyota Land Cruiser  2013, 4.6 L, 87 000 km  Bakı, 22.03.2023 11:20  ,    3 400  AZN    Opel Vectra  1996, 2.0 L, 256 666 km  Sumqayıt, bugün 02:32  ,    6 600  AZN    Volkswagen Passat  2004, 1.8 L, 385 972 km  Bakı, 02.04.2023 16:11  ,    15 100  AZN    Hyundai i30  2008, 1.6 L, 187 000 km  Bakı, 29.03.2023 22:10  ,    27 000  $    Toyota Prado  2012, 2.7 L, 130 000 km  Bakı, 23.03.2023 14:05  ,    28 500  AZN    Hyundai ix35  2014, 2.0 L, 160 000 km  Bakı, 22.03.2023 01:30  ,    12 200  AZN    Toyota Corolla  2

In [8]:
turbo=pd.DataFrame()

In [9]:
for page_num in range(10,20):
    url = f'https://turbo.az/autos?page={page_num}'
    page = requests.get(url)
    html = BeautifulSoup(page.content, 'html.parser')
    cars = html.find_all('div', class_ = 'products-i__bottom')

    for i in range(len(cars)):
        #substite
        carinfo = re.sub('<[^>]+>', '-', str(cars[i])).split('--')

        currency = carinfo[1].split('-')[2]
        price = int(carinfo[1].split('-')[1].replace(' ',''))
        make_model = carinfo[3]
        year = int(carinfo[4].split(',')[0])
        engine = int(carinfo[4].split(',')[1].replace('.','').replace('L','').strip()) * 100
        km = int(carinfo[4].split(',')[2].replace('km','').replace(' ',''))

        turbo = turbo.append({
            'Make_Model':make_model,
            'Production_Year':year,
            'Engine_Power':engine,
            'KM_Driven':km,
            'Price':price,
            'Currency':currency,
        },ignore_index=True)

In [10]:
turbo

,Make_Model,Production_Year,Engine_Power,KM_Driven,Price,Currency
0,Hyundai i30,2011.0,1600.0,186500.0,16800.0,AZN
1,Land Rover Range Rover,2019.0,3000.0,14000.0,110000.0,$
2,Land Rover Range Rover,2013.0,3000.0,162000.0,55900.0,$
3,Kia Sorento,2013.0,2000.0,140000.0,33900.0,AZN
4,Hyundai Elantra,2017.0,1600.0,254000.0,27000.0,AZN
...,...,...,...,...,...,...
355,Opel Vectra,1998.0,1600.0,247652.0,5950.0,AZN
356,Land Rover Range Rover,2008.0,4200.0,98800.0,26000.0,$
357,Fiat Doblo,2016.0,1400.0,21000.0,26500.0,AZN
358,Kia Rio,2021.0,1600.0,39500.0,32400.0,AZN


In [24]:
url_cur='https://www.cbar.az/'

In [25]:
page1=requests.get(url_cur)

In [26]:
html1=BeautifulSoup(page1.content, 'html.parser')

In [27]:
currencies=html1.find_all('div', class_='relize_item')

In [28]:
curr=re.sub('<[^>]+>', ' ', str(currencies))

In [29]:
curr

'[  04.04.2023  Valyuta hərracı barədə  ,   30.03.2023  Valyuta hərracı barədə  ,   29.03.2023  Faiz dəhlizinin parametrləri haqqında  ,   04.04.2023 tarixindən etibarən   1 USD - 1.7000    1 EUR - 1.8519    1 RUB - 0.0217    1 TRY - 0.0885     ,   30.03.2023 tarixindən etibarən   Aşağı həddi - 7.0%    Yuxarı həddi - 9.75%    Uçot faiz dərəcəsi - 8.75%   ]'

In [30]:
curr_usd=float(curr[curr.find('USD')+5:curr.find('1 EUR')].replace(' ', ''))

In [31]:
curr_eur=float(curr[curr.find('EUR')+5:curr.find('1 RUB')].replace(' ', ''))

In [32]:
curr_eur

1.8519

In [33]:
turbo['Currency'].unique()

array(['AZN', '$', '€'], dtype=object)

In [40]:
turbo['Price in AZN']=pd.to_numeric(turbo['Price in AZN'])

In [39]:
turbo['Price in AZN']=np.where(turbo['Currency']=='$', turbo['Price']*curr_usd, np.where(turbo['Currency']=='€',turbo['Price']*curr_eur, turbo['Price']))

In [41]:
turbo

,Make_Model,Production_Year,Engine_Power,KM_Driven,Price,Currency,Price in AZN
0,Hyundai i30,2011.0,1600.0,186500.0,16800.0,AZN,16800.0
1,Land Rover Range Rover,2019.0,3000.0,14000.0,110000.0,$,187000.0
2,Land Rover Range Rover,2013.0,3000.0,162000.0,55900.0,$,95030.0
3,Kia Sorento,2013.0,2000.0,140000.0,33900.0,AZN,33900.0
4,Hyundai Elantra,2017.0,1600.0,254000.0,27000.0,AZN,27000.0
...,...,...,...,...,...,...,...
355,Opel Vectra,1998.0,1600.0,247652.0,5950.0,AZN,5950.0
356,Land Rover Range Rover,2008.0,4200.0,98800.0,26000.0,$,44200.0
357,Fiat Doblo,2016.0,1400.0,21000.0,26500.0,AZN,26500.0
358,Kia Rio,2021.0,1600.0,39500.0,32400.0,AZN,32400.0
